The data are patches from the bbbc021 dataset. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import re
import sys
import pandas as pd
from __future__ import division

from skimage import io, filters, util, data, img_as_float
import microscopium as mic
#import microscopium.features as micfeat
import scipy
import brewer2mpl

from sklearn.cluster import KMeans
from sklearn.decomposition import MiniBatchDictionaryLearning, SparseCoder, sparse_encode, PCA
from sklearn.feature_extraction.image import extract_patches_2d, PatchExtractor, reconstruct_from_patches_2d
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.datasets import make_sparse_coded_signal
from sklearn.manifold import TSNE

#os.getcwd()
os.chdir("/Users/don/Documents/hcs")
sys.path.append("/Users/don/Documents/PyModules")

#Tell numpy to skip division by zero in broadcasting
np.seterr(divide = 'ignore', invalid = 'ignore')

import skynet.pipeline
import skynet.utils
import skynet.rsnn_x as rsnn2

Let's try an RSNN with a bunch of patches and their MOA labels. IO details in bbbc021IO.ipynb.

In [5]:
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
labels_train = np.load('label_train.npy')
labels_test = np.load('label_test.npy')

n_trg, patchlen, patchlen, ch = x_train.shape
n_test, patchlen, patchlen, ch = x_test.shape

print("Data Stats")
print("-"*10)
print("x_train.shape = %s" % (x_train.shape,))
print("x_test.shape = %s" % (x_test.shape,))

Data Stats
----------
x_train.shape = (76960, 30, 30, 3)
x_test.shape = (19240, 30, 30, 3)


Now we'll flatten the data by unravelling each patch, concatenating the colour channels end to end.

In [6]:
patches_train = []
patches_test = []

for i in range(len(x_train)):
    patch_lin = x_train[i].reshape(patchlen*patchlen*ch)
    patches_train.append(patch_lin)
    
for i in range(len(x_test)):
    patch_lin = x_test[i].reshape(patchlen*patchlen*ch)
    patches_test.append(patch_lin)

patches_train = np.array(patches_train)
patches_test = np.array(patches_test)

#Normalization dummy code
"""mydata = mydata.reshape(mydata.shape[0],-1)
print("Data shape after reshape: %s" % (mydata.shape,))

#Normalization
mydata -= np.mean(mydata, axis = 0)
mydata /= np.std(mydata, axis = 0)
mydata[np.isnan(mydata)] = 0.0"""

#To save space
del x_train
del x_test

#Final printout checks
print(patches_train.shape)

(76960, 2700)


And now let's convert the labels to indices. Due to the way that the images were extracted, there will never be any labels in the training set that isn't present in the testing set, or vice versa. Note that train_rfciw() accepts indices just as integers; it'll do the one-hot encoding for you.<br><br>We'll see that there are 13 labels in all. There were originally 103 MOA labels; I suppose that those without compound-concentration representations were dropped.

In [7]:
labels_ref = list(set(labels_train))
print(len(labels_ref))

labels_train2 = skynet.utils.my_one_hot(labels_train, labels_ref)
labels_test2 = skynet.utils.my_one_hot(labels_test, labels_ref)

#print(labels_train2.shape)
#print(labels_test2.shape)

13
(76960,)
(19240,)


In [10]:
t0 = time.time()
w_random, w_out = rsnn2.train_rfciw(patches_train, 
                                    labels_train2, 
                                    n_ch = 3,
                                    n_hidden = 5000, verbose=True)

dt = time.time() - t0
m, s = divmod(dt, 60)
h, m = divmod(m, 60)
print("Done in %d:%02d:%02d" % (h, m, s))
print("w_out.shape = %s" % (w_out.shape,))

TRAINING DATA:
------------
n_images = 76960
img size = 2700
side length = 30.0
n_hidden = 5000

rf.shape = (5000, 2700)

GENERATING RANDOM WEIGHTS
--------------------------
w_random.shape = (5000, 2701)

ACTIVATING INPUT DATA
--------------------
A = f(w_random * X.T) = f(Z)
w_random.shape = (5000, 2701)
X.shape = (76960, 2701)
activations.shape = (5000, 76960)

Done in 0:03:50
w_out.shape = (5000, 13)


In [11]:
# Test accuracy
predict_out = rsnn2.test_accuracy(patches_test,labels_test2,w_random, w_out)
predict_out

0.62281704781704783

That's a fairly impressive 62.28% prediction accuracy. For posterity, this was done with 5000 hidden units, and took 3 mins 50s. <br>

Now let's test our weights against randomly-generated labels, to establish a baseline accuracy of randomly guessing labels. Also note that since a couple of the labels ('DMSO') is vastly overrepresented, in the data set, we could also say that "all samples belong to DMSO" and still be correct about 37% of the time. 

In [16]:
#Get a DF of the labels
labels_ref = list(set(labels_train))

df = []
for i in range(len(labels_ref)):
    df.append(list(labels_train).count(labels_ref[i]))

df = np.array(df)
total = np.sum(df)
df = df/total

In [18]:
#No. of permutations
B = 50

bootstrapped = np.zeros(B)
for i in range(B):
    t0 = time.time()
    print(Iteration i , end = "")
    labels_nonsense = np.random.choice(len(labels_ref),
                                  len(labels_test2),
                                 p=df)
    bootstrapped[i] = rsnn2.test_accuracy(patches_test,labels_nonsense,w_random, w_out)
    print("done in %.2fs" % (time.time() - t0))


0done in 10.99s
1done in 11.29s
2done in 13.45s
3done in 12.45s
4done in 11.40s
5done in 10.89s
6done in 11.24s
7done in 11.14s
8done in 10.98s
9done in 11.52s
10done in 12.22s
11done in 11.01s
12done in 10.99s
13done in 11.54s
14done in 11.74s
15done in 12.64s
16done in 12.37s
17done in 11.26s
18done in 12.98s
19done in 11.53s
20done in 12.31s
21done in 11.49s
22done in 11.43s
23done in 10.96s
24done in 11.66s
25done in 11.41s
26done in 13.13s
27done in 14.38s
28done in 12.35s
29done in 12.66s
30done in 12.21s
31done in 14.41s
32done in 12.57s
33done in 12.53s
34done in 12.37s
35done in 11.77s
36done in 12.21s
37done in 12.15s
38done in 12.49s
39done in 15.28s
40done in 15.93s
41done in 15.83s
42done in 12.54s
43done in 12.18s
44done in 11.00s
45done in 16.09s
46done in 15.04s
47done in 14.92s
48done in 12.67s
49done in 11.82s


In [19]:
mu = np.mean(bootstrapped)
sigma = np.std(bootstrapped)
print(mu, sigma)

0.348406444906 0.00396717247323


Random guessing gives us a 34.84% (s.d. 0.39) accuracy. Further analysis to come: a k-fold CV of some kind? Which classes have the highest classification accuracy (that is, is this correlated with the relative proportions of labels in the training data)?<br><br>Technical note: generating random numbers seems to take up the bulk of running time.